In [ ]:
pip install flask

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3
import os

app = Flask(__name__)

# Define the path for the SQLite database using current working directory
db_path = os.path.join(os.getcwd(), 'farming.db')

# Initialize the database
def init_db():
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS farm_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                field_name TEXT,
                crop_type TEXT,
                moisture INTEGER,
                ph REAL,
                temperature INTEGER,
                growth_stage TEXT
            )
        ''')
        conn.commit()

# Home route to display data and form
@app.route('/')
def home():
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM farm_data")
        data = cursor.fetchall()
    return render_template('index.html', data=data)

# Route to handle user input
@app.route('/add', methods=['POST'])
def add_data():
    if request.method == 'POST':
        field_name = request.form['field_name']
        crop_type = request.form['crop_type']
        moisture = request.form['moisture']
        ph = request.form['ph']
        temperature = request.form['temperature']
        growth_stage = request.form['growth_stage']

        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()
            cursor.execute(
                "INSERT INTO farm_data (field_name, crop_type, moisture, ph, temperature, growth_stage) VALUES (?, ?, ?, ?, ?, ?)",
                (field_name, crop_type, moisture, ph, temperature, growth_stage)
            )
            conn.commit()

        return redirect(url_for('home'))

# Explicitly defining entry point for Flask app
if __name__ == '__main__':
    init_db()  # Initialize the database if it doesn't exist
    app.run(debug=True, use_reloader=False)
